In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de base 
base_url = "https://www.tayara.tn/fr/ads/c/Immobilier/Maisons%20et%20Villas/?page="


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}


data = []

# Fonction pour récupérer les annonces d'une page
def get_page_data(url):
    response = requests.get(url, headers=headers)
    
    # Vérifier si la requête a réussi
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Trouver tous les articles
        articles = soup.find_all('article', class_='mx-0')
        
        # Parcourir chaque article pour obtenir son lien et les infos
        for article in articles:
            link_tag = article.find('a')
            if link_tag and 'href' in link_tag.attrs:
                article_url = "https://www.tayara.tn" + link_tag['href']  # URL complet
                article_response = requests.get(article_url)
                
                if article_response.status_code == 200:
                    article_soup = BeautifulSoup(article_response.text, 'html.parser')
                    
                    # Chercher l'élément <data> contenant le prix, basé sur l'attribut 'value'
                    prix_element = article_soup.find('data', {'value': True})
                    prix = prix_element['value'] if prix_element else None
                    
                    # Extraire les informations sur les critères
                    criteria_section = article_soup.find('h2', class_='text-gray-700 font-semibold text-xl mb-4')
                    if criteria_section:
                        criteria_items = criteria_section.find_next('ul').find_all('li')
                        
                        # Initialisation des variables
                        superficie = None
                        salles_de_bains = None
                        chambres = None
                        transaction_type = None
                        
                        for item in criteria_items:
                            if 'Type de transaction' in item.get_text():
                                transaction_type = item.find('span', class_='text-gray-700/80').text.strip()
                            elif 'Superficie' in item.get_text():
                                superficie = item.find('span', class_='text-gray-700/80').text.strip()
                            elif 'Salles de bains' in item.get_text():
                                salles_de_bains = item.find('span', class_='text-gray-700/80').text.strip()
                            elif 'Chambres' in item.get_text():
                                chambres = item.find('span', class_='text-gray-700/80').text.strip()

                    # Extraction des informations sur la localisation et le type de bien
                    
                    localisation_elements = article_soup.find_all('li', class_='p-2 my-1 text-xs text-gray-600')
                    if localisation_elements:
                        # Extraire les informations de localisation
                        type_bien = localisation_elements[1].text.strip() if len(localisation_elements) > 1 else None
                        ville = localisation_elements[2].text.strip() if len(localisation_elements) > 2 else None
                        place_exacte = localisation_elements[3].text.strip() if len(localisation_elements) > 3 else None
                    else:
                        type_bien = ville = place_exacte = None
                    
                    # Ajouter les informations dans la liste data 
                    data.append({
                        "Type de bien": type_bien,
                        "Ville": ville,
                        "Place exacte": place_exacte,
                        "Superficie (m²)": superficie,
                        "Chambres": chambres,
                        "Salles de bains": salles_de_bains,
                        "Type de transaction": transaction_type,
                        "Prix (DT)": prix
                    })
                else:
                    print(f"Erreur lors de la récupération des détails pour {article_url}")
            else:
                print("Aucun lien trouvé pour cet article.")
    else:
        print(f"Erreur lors du chargement de l'URL {url}")

# Récupérer les données pour les pages
for page_num in range(1, 40):  
    print(f"Récupération des données pour la page {page_num}...")
    url = base_url + str(page_num)
    get_page_data(url)


df = pd.DataFrame(data)

# Afficher les données 
print(df)


Récupération des données pour la page 1...
Récupération des données pour la page 2...
Récupération des données pour la page 3...
Récupération des données pour la page 4...
Récupération des données pour la page 5...
Récupération des données pour la page 6...
Récupération des données pour la page 7...
Récupération des données pour la page 8...
Récupération des données pour la page 9...
Récupération des données pour la page 10...
Récupération des données pour la page 11...
Récupération des données pour la page 12...
Récupération des données pour la page 13...
Récupération des données pour la page 14...
Récupération des données pour la page 15...
Récupération des données pour la page 16...
Récupération des données pour la page 17...
Récupération des données pour la page 18...
Récupération des données pour la page 19...
Récupération des données pour la page 20...
Récupération des données pour la page 21...
Récupération des données pour la page 22...
Récupération des données pour la page 23.